In [1]:
# Adjust working directory to allow importing modules from parent directory
from pathlib import Path
import os

# Get current working directory only if it has not been defined (first run)
try:
    current_directory
except:
    current_directory = Path.cwd()

# Change the working directory to the parent
parent_directory = current_directory.parent
os.chdir(parent_directory)

In [2]:
# Import
from src.utils.preprocessing_tools import *

In [7]:
accommodation_reviews("train")
accommodation_reviews("val")
accommodation_reviews("test")

In [4]:
concatenate_train_val()

train concatenated
val concatenated


In [17]:
# Get processed train
train_df = csv_to_dataframe("train")
print(train_df.shape)
train_df.head()

(1628989, 19)


,user_id,accommodation_id,guest_type,guest_country,room_nights,month,accommodation_type,accommodation_country,accommodation_score,accommodation_star_rating,location_is_ski,location_is_beach,location_is_city_center,review_id,review_title,review_positive,review_negative,review_score,review_helpful_votes
0,8e2ee00e-08bf-4229-aefb-74224a5adfa6,-1109473678,Family with children,Vey,2,2,Hotel,Turkey,8.3,5.0,0,0,0,ca87eb9f-3b2a-4e3b-9f86-6a8d14722a9e,Amazing hotel &amp; staff,I loved the hotel the rooms were amazing and t...,Nothing,10.0,0
1,776accc3-9f1b-4a2e-8616-aafcac7eeb1d,-1189343073,Solo traveller,Gobuf,2,3,Hotel,Italy,8.0,4.0,0,0,0,b555f9d4-176a-4813-b88d-54e0b676be9c,Spacious and Clean Room,Room was very spacious and clean. Staff were f...,Area of the hotel wasn&#39;t great but it was ...,8.0,0
2,e32f90a1-1580-4a87-86a1-7602d5543a15,-1454980525,Couple,Qehoj,3,6,Hotel,Germany,8.5,4.0,0,0,1,baf7a395-049f-4ea7-a380-58e9836b5da2,A wonderful hotel where you feel comfortable a...,"The room was beautifully decorated, the bed wa...",NaN,10.0,0
3,9f246c70-def0-4b93-8c8a-296f5014f6fc,-773005129,Family with children,Mejok,1,8,Apartment,United Kingdom,9.3,0.0,0,0,0,a29ebd72-c49e-4a4c-a68e-093d88c87cf3,NaN,What an amazing apartment!!!!! Easy check in. ...,Nothing,10.0,0
4,2958080f-b80d-4bcb-84e9-2e053c314e38,462909752,Family with children,Zuc,6,6,Hotel,Australia,8.7,4.0,0,1,0,da5f55ce-43c3-4ce5-8f7c-3ef02bedcf68,The Palmere Collective has everything you need...,"We were up for a Cowboys game, and the locatio...","The Hotel could benefit from a bigger, better ...",10.0,0


Remove below (only for fast testing)

In [6]:
# Get first five rows of the dataframe
train_df = train_df.head()

In [21]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# get vectors for the titles
title_vectors = model.encode(train_df["review_title"].values, show_progress_bar=True)
# get vector for positive reviews
positive_review_vectors = model.encode(train_df["positive_review"].values, show_progress_bar=True)
# get vector for negative reviews
negative_review_vectors = model.encode(train_df["negative_review"].values, show_progress_bar=True)

Batches:   0%|          | 0/50906 [00:00<?, ?it/s]

In [15]:
# add the title_vectors to the dataframe
train_df["title_vectors"] = title_vectors.tolist()

C:\Users\t-yzelinger\AppData\Local\Temp\ipykernel_14124\2757780043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["title_vectors"] = title_vectors.tolist()


In [16]:
train_df

,user_id,accommodation_id,guest_type,guest_country,room_nights,month,accommodation_type,accommodation_country,accommodation_score,accommodation_star_rating,location_is_ski,location_is_beach,location_is_city_center,review_id,review_title,review_positive,review_negative,review_score,review_helpful_votes,title_vectors
0,8e2ee00e-08bf-4229-aefb-74224a5adfa6,-1109473678,Family with children,Vey,2,2,Hotel,Turkey,8.3,5.0,0,0,0,ca87eb9f-3b2a-4e3b-9f86-6a8d14722a9e,Amazing hotel &amp; staff,I loved the hotel the rooms were amazing and t...,Nothing,10.0,0,"[-0.03325358405709267, -0.01957494020462036, -..."
1,776accc3-9f1b-4a2e-8616-aafcac7eeb1d,-1189343073,Solo traveller,Gobuf,2,3,Hotel,Italy,8.0,4.0,0,0,0,b555f9d4-176a-4813-b88d-54e0b676be9c,Spacious and Clean Room,Room was very spacious and clean. Staff were f...,Area of the hotel wasn&#39;t great but it was ...,8.0,0,"[-0.02749762311577797, 0.01642598770558834, -0..."
2,e32f90a1-1580-4a87-86a1-7602d5543a15,-1454980525,Couple,Qehoj,3,6,Hotel,Germany,8.5,4.0,0,0,1,baf7a395-049f-4ea7-a380-58e9836b5da2,A wonderful hotel where you feel comfortable a...,"The room was beautifully decorated, the bed wa...",NaN,10.0,0,"[-0.02371993288397789, 0.002632330870255828, -..."
3,9f246c70-def0-4b93-8c8a-296f5014f6fc,-773005129,Family with children,Mejok,1,8,Apartment,United Kingdom,9.3,0.0,0,0,0,a29ebd72-c49e-4a4c-a68e-093d88c87cf3,NaN,What an amazing apartment!!!!! Easy check in. ...,Nothing,10.0,0,"[-0.02318720705807209, 0.05149747058749199, -0..."
4,2958080f-b80d-4bcb-84e9-2e053c314e38,462909752,Family with children,Zuc,6,6,Hotel,Australia,8.7,4.0,0,1,0,da5f55ce-43c3-4ce5-8f7c-3ef02bedcf68,The Palmere Collective has everything you need...,"We were up for a Cowboys game, and the locatio...","The Hotel could benefit from a bigger, better ...",10.0,0,"[-0.04188578203320503, -0.013479542918503284, ..."
